# 要約 
このJupyter Notebookは、Kaggleの「LLM 20 Questions」コンペティションに参加するためのボットを作成する内容です。このボットは、質問者として動作し、以下の簡単な戦略を用いてターゲットワードを推測します。

### 問題の内容
コンペティションのルールに従い、ボットが対象物を推測するために「はい」と「いいえ」で回答する質問を行うゲーム形式のアプローチを取っています。今回のボットは、基本的な質問フォーマットやランダムな単語の推測方法を使用しています。

### 手法とライブラリ
1. **基本的なアプローチ**:
   - ボットは常に「それは物ですか、それとも場所ですか？」という質問から始めます。
   - ターゲットワードの推測は、`keywords.py`に含まれるランダムな単語を選ぶことで行います。
   - 回答者には、常に「いいえ」と返答させるよう設計されています。

2. **使用するライブラリ**:
   - **os, sys, glob, random, json**: これらのライブラリを用いて、ファイル操作やランダムな選択、JSON形式のデータ処理を行っています。
   - **numpy、pandas**: 最終的にはファイルパスの表示や管理に使われています。

3. **コードの主な構成**:
   - ファイル管理: `submission` ディレクトリを作成し、必要なファイルをコピー。
   - ボットの応答ロジックを含む `main.py` を作成し、ディレクトリ内のファイルを圧縮化して提出用のファイルにしています。

### まとめ
このNotebookは、シンプルかつ基本的な戦略で作られたボットを提案しており、初学者でも理解しやすい構成としています。シンプルな設問と固定的な回答を用いて、コンペティションにおけるボットの作成手法を示しています。コメントセクションには、他の参加者からの感謝の意が寄せられ、初心者にとっての有益さが強調されています。

---


# 用語概説 
以下は、初心者がノートブックの内容でつまずく可能性のある、マイナーな専門用語に関する簡単な解説です。

1. **LB（Leaderboard）**: コンペティションにおけるボットのパフォーマンスを示すスコア。各ボットは他のボットと対戦して得た結果に基づいてランキングされ、LBはその順位に関連する数値を指します。

2. **Kaggle**: データサイエンスのコンペティションを提供するプラットフォーム。参加者が様々なデータセットを用いてアルゴリズムを開発し、評価します。

3. **ボット**: 特定のタスクを自動で実行するプログラム。ここでは「20の質問」ゲームに挑戦するために設計されたエージェントを指します。

4. **ランダムな単語**: ボットが質問や推測をする際に、予測や回答の選択肢として無作為に選ばれる単語。ゲームの戦略を評価するために使われます。

5. **KEYWORDS_JSON**: `keywords.py`からのデータ構造。ここには、ゲームで使う単語のリストが含まれており、ボットが推測に利用します。

6. **エージェントの関数**: ボットが具体的な動作を行うために定義された関数。例えば、質問をする、推測する、回答するなどの処理が行われます。

7. **obs（観察オブジェクト）**: ボットが現在のゲームの状態を表すオブジェクト。ゲームの進捗に応じて、次の動作を決定するために利用されます。

8. **cfg（設定オブジェクト）**: エージェントの動作に関する設定情報を含むオブジェクト。具体的には、ゲームのルールや制限などが設定されていることが多いです。

9. **パース（parse）**: データの構文を解析して、意味のある構造に変換すること。ここでは、JSON形式のデータをPythonで扱うために用いられます。

10. **圧縮ファイル**: 複数のファイルを一つにまとめ、サイズを小さくするための形式。コンペティションに提出する際に必要なファイルを整理しやすくするために使用されます。

これらの用語は、初心者がノートブックの内容を理解するために役立つ基礎的な知識を補完するものです。

---


# 最も簡単な提出物（LB: ~651.4）
`/kaggle/working/submission.tar.gz` にボットを書き込みます。このボットは以下を行います：
- いつも**場所か物か**を尋ねる
- `keywords.py` からランダムな単語を**推測**する
- いつも**いいえ**と答える

Gemmaモデルは現在、安定性に欠けるようで、LBは約635.5を推移しています。

In [ ]:
# 作業ディレクトリに移動します
!cd /kaggle/working
# 'submission'という名前の新しいディレクトリを作成します
!mkdir submission
# 現在のディレクトリ内のすべてのファイルの完全なパスを表示します
!readlink -f *

In [ ]:
# 'keywords.py'というファイルをコピーします
# /kaggle/input/llm-20-questions/llm_20_questions/ から 'submission' ディレクトリにコピーします
!cp /kaggle/input/llm-20-questions/llm_20_questions/keywords.py submission/keywords.py

In [ ]:
%%writefile submission/main.py

# 必要なライブラリをインポートします
import os, sys, glob, random, json

# keywords.py から KEYWORDS_JSON をインポートします
from keywords import KEYWORDS_JSON
# KEYWORDS_JSON をパースして、最初の要素を取得します
kws = json.loads(KEYWORDS_JSON)[0]

# エージェントの関数を定義します
def agent_fn(obs, cfg):
    # 基本的な質問をする場合の応答
    if obs.turnType == "ask": 
        response = 'それは物ですか、それとも場所ですか？'
    # 推測する場合の応答
    elif obs.turnType == "guess": 
        response = random.choice(kws['words'])['keyword']  # keywordsからランダムに単語を選びます
    # 回答する場合の応答
    elif obs.turnType == "answer": 
        response = 'いいえ'
    return response  # 応答を返します

In [ ]:
# 作成した 'submission' ディレクトリ内のすべてのファイルを圧縮します
# submission.tar.gz という名前で圧縮ファイルを作成します
!tar czf submission.tar.gz -C /kaggle/working/submission .

In [ ]:
# 現在のディレクトリ内のファイルとフォルダをリスト表示します
!ls
# 'submission' ディレクトリ内のファイルとフォルダをリスト表示します
!ls submission

In [ ]:
# numpy と pandas をインポートします
import numpy as np
import pandas as pd
import os

# /kaggle ディレクトリ内のすべてのファイルパスを表示するために os.walk を使用します
for dirname, _, filenames in os.walk('/kaggle'):
    for filename in filenames:
        # 各ファイルのフルパスを印刷します
        print(os.path.join(dirname, filename))

In [ ]:
# 現在のディレクトリ内のファイルとフォルダをリスト表示します
!ls
# submission.tar.gz を展開します
!tar -xzvf submission.tar.gz 
# main.py の内容を表示します
!cat main.py
# 展開後の現在のディレクトリ内のファイルとフォルダを再度リスト表示します
!ls

---

# コメント 

> ## Yangtze Hao
> 
> 私のような初心者にはとても役立ちます。ありがとうございます。
> 
> 

---

> ## Jonathan Harker
> 
> ありがとう。この情報で、競技会のエントリーポイントと出力の期待されるフォーマットを理解するのに役立ちました。非常に便利です！
> 
> 

---